In [1]:
import os

import numpy as np

import torch as T
import torch.nn as nn
import torch.nn.functional as F #for the activation functions? 
import torch.optim as optim 

from make_env import make_env

ModuleNotFoundError: No module named 'torch'

In [21]:
env = make_env('simple_adversary')

In [19]:
env = make_env('simple_adversary')
print(env.n)
#indexing is which agent we want? 
print(env.observation_space)
#discrete? but we need cont 
print(env.action_space)
#0 is do nothing and the rest is 'wasd'
print(env.action_space[0].n)

observation = env.reset()

print(observation)

3
[Box(8,), Box(10,), Box(10,)]
[Discrete(5), Discrete(5), Discrete(5)]
5
[array([ 0.37354364, -1.17757323,  0.91507418, -1.71695189,  1.06683316,
       -1.11813814,  0.92040054, -0.91627065]), array([-0.15175897, -0.59881375, -0.69328952, -0.05943509, -0.15175897,
       -0.59881375, -1.06683316,  1.11813814, -0.14643261,  0.2018675 ]), array([-0.00532636, -0.80068125, -0.5468569 , -0.26130258, -0.00532636,
       -0.80068125, -0.92040054,  0.91627065,  0.14643261, -0.2018675 ])]


In [45]:
class MultiAgentReplayBuffer:
    
    def __init__(self, max_size, critics_dims, actor_dims, n_actions, n_agents, batch_size):
        
        self.mem_size = max_size
        self.mem_cntr = 0
        self.n_agents = n_agents
        self.batch_size = batch_size
        self.n_actions = n_actions
        self.actor_dims = actor_dims
        #self.critic_dims = critic_dims
        
        #critic/actor dims? 
        
        #state memory of a critic? 
        self.state_memory = np.zeros((self.mem_size, critic_dims))
        self.new_state_memory = np.zeros((self.mem_size, critic_dims))
        
        self.reward_memory = np.zeros((self.mem_size, n_agents))
        self.terminal_memory = np.zeros((self.mem_size, n_agents), dtype=bool)
        
        self.init_actor_memory()
        
    def init_actor_memory(self):
        
        self.actor_state_memory = []
        self.actor_new_state_memory = []
        
        self.actor_action_memory = []
        
        for i in range(n_agents): 
            
            self.actor_state_memory.append(np.zeros((self.mem_size, self.actor_dims[i])))
            self.actor_new_state_memory.append(np.zeros((self.mem_size, self.actor_dims[i])))
            
            self.actor_action_memory.append(np.zeros((self.mem_size, self.n_actions)))
            
    def store_transition(self, raw_obs, state, action, reward, raw_obs_, state_, done): 
        
        if self.mem_cntr % self.mem_size == 0 and self.mem_cntr > 0 : 
            #when we reach the end of a memory we want to override the earliest memories 
            #buggy fixed in github? 
            self.init_actor_memory()
            
        #postiion of the first available memory
        index = self.mem_cntr % self.mem_size
        
        for agent_idx in range(self.n_agents) :
            
            self.actor_state_memory[agent_idx][index] = raw_obs[agent_idx]
            self.actor_new_state_memory[agent_idx][index] = raw_obs_[agent_idx]
            self.actor_action_memory[agent_idx][index] = action[agent_idx]
            
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done
        
        self.mem_cntr += 1 
        
    def sample_buffer(self): 
        
        #what is the highest positon we filled in our memory? 
        #...because we don't want to sample a bunch of zeros
        
        max_mem = min(self.mem_cntr, self.mem_size)
        
        batch = random.choice(max_mem, self.batch_size, replace = False)
        #batch = random.choice(max_mem, self.batch_size)
        
        states = self.state_memory[batch]
        rewards = self.reward_memory[batch]
        states_ = self.new_state_memory[batch]
        terminal = self.terminal_memory[batch]
        
        actor_states = []
        actor_new_states = []
        actions = []
        
        for agent_idx in range(self.n_agents): 
            
            actor_states.append(self.actor_state_memory[agent_idx][batch])
            actor_new_states.append(self.actor_new_state_memory[agent_idx][batch])
            actions.append(self.actor_action_memory[agent_idx][batch])
            
        #difference between actor_states and states is that 
        #..actor_states are individual numpy arrays of length wither 8 or 10 or 10
        #..and the states are the flatenned combination all three of those
        #..that we use to pass into our critic 
        #.. and the actor states are what we use to pass to each individual actor for each agent
        
        return actor_states, states, actions, rewards, actor_new_states, states_, terminal
            
            
    #function to determine weather we are allowed to sample a memory based on weather or not
    #..we have filled up the batch size of our memories 
    
    def ready(self):
        if self.mem_cntr >= self.batch_size: 
            return True 
        return False

In [24]:
class CriticNetwork(nn.Module): 
    
    #here we use the same learning rate for actor and critic - beta, but we can use separate ones
    # fc - fully connected layers POG 
    
    def __init__(self, beta, input_dims, fc1_dims, fc2_dims, n_agents, n_actions, name, chkpt_dir): 
        
        super(CriticNetwork, self).__init__()
        
        self.chkpt_file = os.path.join(chkpt_dir, name)
        
        #our Critic is taking in full state observation vector of the whole system (28 dimensions - 8,10,10)
        #..and then we have to feed in the full action vectors for each agent
        #input dims is the full state vector size? input_dims = 28
        #n_agents(3)*n_actions(5) = 15
        
        self.fc1 = nn.Linear(input_dims+n_agents*n_actions, fc1_dims)
        self.fc2 = nn.Linear(fc1_dims, fc2_dims)
        self.q = nn.Linear(fc2_dims, 1)
        
        self.optimizer = optim.Adam(self.parameters(), lr = beta)
        
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, state, action):
        
        #cat? 
        x = F.relu(self.fc1(T.cat([state, action], dim=1)))
        x = F.relu(self.fc2(x))
        q = self.q(x)
        
        return q
    
    def save_checkpoint(self):
        T.save(self.state_dict(), self.chkpt_file)
        
    
    def load_checkpoint(self): 
        self.load_state_dict(T.load(self.chkpt_file))
         

In [29]:
class ActorNetwork(nn.Module):
    def __init__(self, alpha, input_dims, fc1_dims, fc2_dims, n_actions, name, chkpt_dir): 
        super(ActorNetwork, self).__init__()
        
        self.chkpt_file = os.path.join(chkpt_dir, name)
        
        #input dims here are either 8 or 10, so not 28 
        self.fc1 = nn.Linear(input_dims, fc1_dims)
        self.fc2 = nn.Linear(fc1_dims, fc2_dims)
        self.pi = nn.Linear(fc2_dims, n_actions)
        
        self.optimizer = optim.Adam(self.parameters(), lr = alpha)
        
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, state): 
        
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        pi = T.softmax(self.pi(x), dim = 1)
        
        return pi
    
    def save_checkpoint(self):
        T.save(self.state_dict(), self.chkpt_file)
        
    def load_checkpoint(self): 
        self.load_state_dict(T.load(self.chkpt_file))
        
    
        

In [26]:
class Agent:
    def __init__(self, actor_dims, critic_dims, n_actions, agent_idx, chkpt_dir,
                 alpha=0.01, beta=0.01, fc1=64, fc2=64, gamma = 0.95, tau = 0.01): 
        self.gamma = gamma
        self.tau = tau
        self.n_actions = n_actions 
        self.agent_name = 'agent_%s' % agent_idx 
        
        self.actor = ActorNetwork(alpha, actor_dims, fc1, fc2, 
                                  n_actions, name= self.agent_name + '_actor', chkpt_dir=chkpt_dir)
        self.critic = CriticNetwork(beta, critic_dims, fc1, fc2, n_agents,
                                    n_actions, name= self.agent_name + '_critic', chkpt_dir=chkpt_dir)
        
        self.target_actor = ActorNetwork(alpha, actor_dims, fc1, fc2, n_actions, 
                                         name= self.agent_name + '_target_actor', chkpt_dir=chkpt_dir)
        self.target_critic = CriticNetwork(beta, critic_dims, fc1, fc2, n_agents, n_actions, 
                                           name=self.agent_name + '_target_critic', chkpt_dir=chkpt_dir)
        
        self.update_network_parameters(tau=1)
        
    def update_network_parameters(self, tau=None):
        
        if tau is None:
            tau = self.tau
        
        #actor
        
        target_actor_params = self.target_actor.named_parameters()
        actor_params = self.actor.named_parameters()
        
        target_actor_state_dict = dict(target_actor_params)
        actor_state_dict = dict(actor_params)
        
        for name in actor_state_dict:
            actor_state_dict[name] = tau*actor_state_dict[name].clone() + (1-tau)*target_actor_state_dict[name].clone()
        
        self.target_actor.load_state_dict(actor_state_dict)
        
        #critic
        
        target_critic_params = self.target_critic.named_parameters()
        critic_params = self.critic.named_parameters()
        
        target_critic_state_dict = dict(target_critic_params)
        critic_state_dict = dict(critic_params)
        
        for name in critic_state_dict:
            critic_state_dict[name] = tau*critic_state_dict[name].clone() + (1-tau)*target_critic_state_dict[name].clone()
        
        self.target_critic.load_state_dict(critic_state_dict)
            
    
    def choose_action(self, observation):
        #we are turning our observation into a pytorch tensor
        state = T.tensor([observation], dtype = T.float).to(self.actor.device)
        actions = self.actor.forward(state)
        noise = T.rand(self.n_actions).to(self.actor.device)
        action = actions + noise
        
        return action.detach().cpu().numpy()[0]
    
    
    def save_models(self):
        
        self.actor.save_checkpoint()
        self.target_actor.save_checkpoint()
        self.critic.save_checkpoint()
        self.target_critic.save_checkpoint()
        
    
    
    def load_models(self):
        
        self.actor.load_checkpoint()
        self.target_actor.load_checkpoint()
        self.critic.load_checkpoint()
        self.target_critic.load_checkpoint()
        


In [35]:
#MADDPG class - wrapper class that handles the fact that we have multiple agents 

class MADDPG:
    
    #creating a list of agents and calling the constructor for each agent
    
    def __init__(self, actor_dims, critic_dims, n_agents, n_actions, 
                scenario = 'simple', alpha=0.01, beta=0.01, fc1=64, fc2=64, 
                gamma = 0.99, tau = 0.01, chkpt_dir = 'tmp/maddpg/'):
        
        self.agents = []
        self.n_agents = n_agents
        self.n_actions = n_actions 
        chkpt_dir += scenario 
        
        for agent_idx in range(self.n_agents):
            
            self.agents.append(Agent(actor_dims[agent_idx], critic_dims, n_actions, agent_idx, alpha=alpha, beta=beta, chkpt_dir=chkpt_dir))
            
    def save_checkpoint(self):
        
        print('...saving checkpoint...')
        for agent in self.agents: 
            agent.save_models()
            
    def load_checkpoint(self):  
        print('...loading checkpoint...')
        for agent in self.agents: 
            agent.load_models()       
        
        
    #env expects a list of actions to the step function?
    def choose_action(self, raw_obs):
        actions = []
        for agent_idx, agent in enumerate(self.agents):
            action = agent.choose_action(raw_obs[agent_idx])
            actions.append(action)
        return actions
        
    #memory will be a global memory replay buffer?
    def learn(self, memory):
        if not memory.ready():
            return
        
        actor_states, states, actions, rewards, \
        actor_new_states, states_, dones = memory.sample_buffer()
        
        device = self.agents[0].actor.device
        states = T.tensor(states, dtype=T.float).to(device)
        actions = T.tensor(actions, dtype=T.float).to(device)
        rewards = T.tensor(rawards, dtype=T.float).to(device)
        states_ = T.tensor(states_, dtype=T.float).to(device)
        dones = T.tensor(dones).to(device)
        
        all_agents_new_actions = []
        all_agents_new_mu_actions = []
        old_agents_actions = []
        
        for agent_idx, agent in enumerate(self.agents): 
            
            #we need all three different actions (actions according to the target network for the new states, 
            # ..., actions according to the actor network for the current states, 
            # the actions the agent actually took). 
            #we need them for the calculations of the loss function
            
            new_states = T.tensor(actor_new_states[agent_idx], dtype=T.float).to(device)
            new_pi = agent.target_actor.forward(new_states)
            all_agents_new_actions.append(new_pi)
            
            mu_states = T.tensor(actor_states[agent_idx], dtype=T.float).to(device)
            pi = agent.actor.forward(mu_states)
            all_agents_new_mu_actions.append(pi)
            
            old_agents_actions.append(actions[agent_idx])
            
            new_actions = T.cat([acts for acts in all_agents_new_actions], dim=1)
            mu = T.cat([acts for acts in all_agents_new_mu_actions], dim=1)
            old_actions = T.cat([acts for acts in old_agents_actions], dim=1)
            
            for agent_idx, agent in enumerate(self.agents):
                
                critic_value_ = agent.target_critic.forward(states_, new_actions).flatten()
                critic_value_[dones[:, 0]] = 0.0
                critic_value = agent.critic.forward(states, old_actions).flatten()
                
                #all rows for this particular agent 
                target = rewards[:, agent_idx] + agent.gamma*critic_value_
                
                critic_loss = F.mse_loss(target, critic_value)
                
                agent.critic.optimizer.zero_grad()
                critic_loss.backward(retain_graph=True)
                agent.critic.optimizer.step()
                
                #mu are the actions for the current states according to the regulart actor network -
                # what action should we take given the new values of our actor network 
                # as opposed to the actions we actually took 
                
                actor_loss = agent.critic.forward(states, mu).flatten()
                actor_loss = -T.mean(actor_loss)
                agent.actor.optimizer.zero_grad()
                actor_loss.backward(retain_graph=True)
                agent.actor.optimizer.step()
                
                agent.update_network_parameters()
                



In [44]:
import random

def obs_list_to_state_vector(observation):
    state = np.array([])
    for obs in observation:
        state = np.concatenate([state, obs])
    return state

if __name__ == '__main__':
    #scenario = 'simple'
    scenario = 'simple_adversary'
    env = make_env(scenario)
    n_agents = env.n
    actor_dims = []
    for i in range(n_agents):
        actor_dims.append(env.observation_space[i].shape[0])
    critic_dims = sum(actor_dims)

    # action space is a list of arrays, assume each agent has same action space
    n_actions = env.action_space[0].n
    maddpg_agents = MADDPG(actor_dims, critic_dims, n_agents, n_actions, 
                           fc1=64, fc2=64,  
                           alpha=0.01, beta=0.01, scenario=scenario,
                           chkpt_dir='tmp/maddpg/')

    memory = MultiAgentReplayBuffer(1000000, critic_dims, actor_dims, 
                        n_actions, n_agents, batch_size=1024)

    PRINT_INTERVAL = 500
    N_GAMES = 50000
    MAX_STEPS = 25
    total_steps = 0
    score_history = []
    evaluate = False
    best_score = 0

    if evaluate:
        maddpg_agents.load_checkpoint()

    for i in range(N_GAMES):
        obs = env.reset()
        score = 0
        done = [False]*n_agents
        episode_step = 0
        while not any(done):
            if evaluate:
                env.render()
                #time.sleep(0.1) # to slow down the action for the video
            actions = maddpg_agents.choose_action(obs)
            obs_, reward, done, info = env.step(actions)

            state = obs_list_to_state_vector(obs)
            state_ = obs_list_to_state_vector(obs_)

            if episode_step >= MAX_STEPS:
                done = [True]*n_agents

            memory.store_transition(obs, state, actions, reward, obs_, state_, done)

            if total_steps % 100 == 0 and not evaluate:
                maddpg_agents.learn(memory)

            obs = obs_

            score += sum(reward)
            total_steps += 1
            episode_step += 1

        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        if not evaluate:
            if avg_score > best_score:
                maddpg_agents.save_checkpoint()
                best_score = avg_score
        if i % PRINT_INTERVAL == 0 and i > 0:
            print('episode', i, 'average score {:.1f}'.format(avg_score))

TypeError: choice() takes 2 positional arguments but 3 were given

In [42]:
#func will take each individual 8, 10 and 10 numpy array and turn it into a single numpy array of
# 28 elements
def obs_list_to_state_vector(observation):
    state = np.array([])
    for obs in observation:
        state = np.concatenate([state, obs])
    return state
        

if __name__ == '__main__':
    scenario = 'simple'
    env = make_env(scenario)
    n_agents = env.n
    actor_dims = []
        
    for i in range(n_agents): 
        actor_dims.append(env.observation_space[i].shape[0])
    #28 for the critic 
    critic_dims = sum(actor_dims)
        
    #we assume that all agents have the same number of actions
    n_actions = env.action_space[0].n
    
    maddpg_agents = MADDPG(actor_dims, critic_dims, n_agents, n_actions, 
                           fc1=64, fc2=64, alpha=0.01, beta=0.01, scenario=scenario,
                          chkpt_dir = 'tmp/maddpg/')
    
    memory = MultiAgentReplayBuffer(1000000, critic_dims, actor_dims, 
                                   n_actions, n_agents, batch_size=1024)
    
    PRINT_INTERVAL = 500
    N_GAMES = 30000
    MAX_STEPS = 25
    total_steps = 0
    score_history = []
    #waether or not we want to evaluate the performance of our agent
    evaluate = False 
    best_score = 0
    
    if evaluate: 
        maddpg_agents.load_checkpoint()
        
    for i in range(N_GAMES): 
        obs = env.reset()
        score = 0
        done = [False]*n_agents
        episode_step = 0
        while not any(done): 
            if evaluate:
                env.render()
            actions = maddpg_agents.choose_action(obs)
            obs_, reward, done, info = env.step(actions)
            
            #??
            state = obs_list_to_state_vector(obs)
            state_ = obs_list_to_state_vector(obs_)
            
            if episode_step > MAX_STEPS: 
                done = True*n_agents
                
            memory.store_transition(obs, state, actions, reward, obs_, state_, done)
            
            if total_steps % 100 == 0 and not evaluate: 
                
                maddpg_agents.learn(memory)
                
            obs = obs_
            
            score += sum(reward)
            total_steps += 1
            episode_step += 1
            
        score_history.append(score)
        #every previous 100 games
        avg_score = np.mean(score_history[-100:])
        if not evaluate: 
            if avg_score > best_score: 
                maddpg_agents.save_checkpoint()
                best_score = avg_score
        if i % PRINT_INTERVAL == 0 and i > 0: 
            print('episode', i, 'average score {:.1f}'.format(avg_score))     
        
    

TypeError: 'int' object is not iterable

In [31]:
if __name__ == '__main__':
    scenario = 'simple'
    env = make_env(scenario)
    n_agents = env.n
    actor_dims = []
        
    for i in range(n_agents): 
        actor_dims.append(env.observation_space[i].shape[0])
    #28 for the critic 
    critic_dims = sum(actor_dims)
        
    #we assume that all agents have the same number of actions
    n_actions = env.action_space[0].n
    
    maddpg_agents = MADDPG(actor_dims, critic_dims, n_agents, n_actions, 
                           fc1=64, fc2=64, alpha=0.01, beta=0.01, scenario=scenario,
                          chkpt_dir = 'tmp/maddpg/')
    
    memory = MultiAgentReplayBuffer(1000000, critic_dims, actor_dims, 
                                   n_actions, n_agents, batch_size=1024)
    
    PRINT_INTERVAL = 500
    N_GAMES = 30000
    MAX_STEPS = 25
    total_steps = 0
    score_history = []
    #waether or not we want to evaluate the performance of our agent
    evaluate = False 
    best_score = 0
    
    if evaluate: 
        maddpg_agents.load_checkpoint()
        
    for i in range(N_GAMES): 
        obs = env.reset()
        score = 0
        done = [False]*n_agents
        episode_step = 0
        while not any(done): 
            if evaluate:
                env.render()
            actions = maddpg_agents.choose_action(obs)
            obs_, reward, done, info = env.step(actions)
            
            #??
            state = maddpg_agents.obs_list_to_state_vector(obs)
            state_ = maddpg_agents.obs_list_to_state_vector(obs_)
            
            if episode_step > MAX_STEPS: 
                done = True*n_agents
                
            memory.store_transition(obs, state, actions, reward, obs_, state_, done)
            
            if total_steps % 100 == 0 and not evaluate: 
                
                maddpg_agents.learn(memory)
                
            obs = obs_
            
            score += sum(reward)
            total_steps += 1
            episode_step += 1
            
        score_history.append(score)
        #every previous 100 games
        avg_score = np.mean(score_history[-100:])
        if not evaluate: 
            if avg_score > best_score: 
                maddpg_agents.save_checkpoint()
                best_score = avg_score
        if i % PRINT_INTERVAL == 0 and i > 0: 
            print('episode', i, 'average score {:.1f}'.format(avg_score))
            
                
            
        

TypeError: obs_list_to_state_vector() takes 1 positional argument but 2 were given